In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn.datasets
from sklearn.linear_model import SGDClassifier

EPOCH = 100
LR = 0.1

# ~ PoC AI Pool 2024 ~
- ## Day 2: Neural Networks from Scratch
    - ### Module 2: Logistic Regression
-----------

Congratulations on building your first machine learning algorithm ! You were probably getting really impatient of diving into AI. I hope you understand why we wanted to take the time to go through all the basics first, though, because as you could probably tell, our python and numpy skills are going to prove really useful when building machine learning models.

During the first module of the day, you got the gist of the main aspects of a machine learning pipeline:
- making a prediction
- computing the loss
- computing the gradients
- updating the weight and bias

You'll find that this basic architecture is behind almost everything we'll be doing for the rest of the week.

You will also find that this basic architecture could be recycled so that the developer can focus entirely on the things that do change.

For example, here's an example of how **Linear Regression** can be achieved using the most popular ML library, **pytorch**:

```python
class LinearRegression(nn.Module):
    def __init__(self):
        self.fc = nn.Linear(***,***)
    def forward(self, x):
        return self.fc(x)
```

And actually, here's the same for **Logistic Regression**, which is what we'll be implementing by hand in this module !

```python
class LogisticRegression(nn.Module):
    def __init__(self):
        self.fc = nn.Linear(***,***)
    def forward(self, x):
        x = self.fc
        return F.sigmoid(x)
```

Cool right ? Well, it might look great for Linear Regression, since you already know what's going on behind the scenes...\
But unless you already know how Logistic Regression works, the code sample won't tell you anything !

That's why we're taking the time to learn the (boring?) math behind these algorithms. It might be annoying at first, but I can assure you that understanding why we use Linear instead of Logistic Regression for certain tasks is much more intuitive if you know how they work than if you have no idea.

First of all, we're going to be using an actual ML library before we begin !

The library is called sklearn and it is a wonderful set of tools which can help while working on AI !

In fact, sklearn has implementations of many algorithms, including Linear and Logistic Regression !

It also provides us with plenty of tools to quickly generate and manipulate randomized data for training :

In [ ]:
## Using `make_blobs()`, we generate a sample dataset with 1_000 entries, each with two features.
## With the `centers` parameter, we tell sklearn to separate the data in two main classes
## Logistic Regression being a classifier model, we will use it to predict if one data entry
## belongs to one class or the other !
x_train, y_train = sklearn.datasets.make_blobs(n_samples=1_000, n_features=2, centers=2)

## This data doesn't mean anything, like Brad's problem in the last module, but if you're wondering
## how multiple features would translate into a real world problem, imagine if you had data of
## house prices and their size, and you needed to predict whether Brad would be willing to buy the
## house or not. That would mean each data entry would have two features: the price and size of the
## house. The "x" would be an array of [price, size] and "y" would be a binary value (either true or false).
x_train.shape, y_train.shape

We'll use matplotlib to display our data in a nice way :

In [ ]:
plt.scatter(x_train[:,0], x_train[:,1])
plt.show()

Each entry is represented by a blue circle, and you can clearly see that there are two clearly separate groups of data.

Now, we'll use sklearn's `SGDClassifier` to train a logistic regression on our generated data.

If you're curious, you might stumble upon `LogisticRegression` while browsing through [sklearn's docs](https://scikit-learn.org/stable/supervised_learning.html#supervised-learning), which is also a sklearn model which implements the eponymous algorithm.

The reason we use `SGDClassifier` instead is because it adds the notion of gradient descent and updating weights to the basic Logistic Regression algorithm.

>SGD stands for 'stochastic gradient descent' btw

In [ ]:
mdl = SGDClassifier(eta0=LR, max_iter=EPOCH)
mdl.fit(x_train, y_train)

plt.scatter(x_train[:,0], x_train[:,1], c=mdl.predict(x_train))
plt.show()
print(f"{(mdl.predict(x_train) == y_train).mean()*100}% accuracy")

As you can see, sklearn was able to fit a model to our data pretty quickly and get an accuracy extremely close to 100%.\
We've colored our data so that you can clearly see how every entry is classified by the sklearn model.\
Keep this in mind, because we'll compare it to our own model's predictions later.

In order to achieve the same results, let's build our model to be used the same way as sklearn, so we'll have our own `fit()` and `predict()` methods.\
And, just like sklearn, we'll wrap our model inside a class so that it can keep track of its weights and biases.

> Storing all of our model's functionalities inside a class will also allow us to save and load the model's weights and biases easily

Go ahead and create your own `MyLogisticRegression` class :

* It should have an `__init__()` method which receives and stores the amount of epochs (`max_iter`) and the learning rate (`lr`).

In [ ]:
class MyLogisticRegression:
    def __init__(self, max_iter=EPOCH, lr=LR):
        pass

assert MyLogisticRegression().max_iter == EPOCH, "we can't find max_iter inside your class"
assert MyLogisticRegression().lr == LR, "we can't find lr inside your class"

Great, now, let's build our `fit()` method.

> We'd like to start with this method so that you have an overview of which methods must be implemented.\
> So, for now, the method won't work because it will call other methods which are not implemented yet !



In [ ]:
def fit(self, x: np.ndarray, y: np.ndarray, train=True):
    """
    The fit method initialises the weights and biases, then runs the training loop
    in order to fit these weights and biases to the input data.
    """
    ## Initialise the weights and biases to 0
    ## Keep in mind that we have only one layer
    ## While both of these values must be initialised as 0, they won't be initialised the same way
    ## Try to remember the properties and use of both of these values if you can't remember why
    self.w = None
    self.b = None

    assert np.mean(self.w) == 0, "w should be initialised to 0"
    assert self.b == 0, "b should be initialised to 0"

    if train is False:
        return

    ## We'll call the training loop for you here
    ## It is the same as for any other model
    ## What changes for logistic regression are the forward and backward methods
    for i in range(self.max_iter):
        y_pred = self._forward(x)
        loss = self._bce(y_pred, y)
        dw, db = self._backward(x, y)
        self._optimize(dw, db)

MyLogisticRegression.fit = fit
MyLogisticRegression().fit(x_train, y_train, train=False)

Now, let's get the `predict()` method out of the way so that we already have all the visible methods.

In a logistic regression model, the outputs, instead of being a prediction for a new value, are a prediction of probabilities.

In a binary classification, the model has one output value which is the probability of the input belonging to one class or the other.

<img width="50%" src="assets/image-9.png"/>
<br>
<br>
In the picture above, the model's first output is `0.6`. This means that the model's prediction is `1`.
<br>
<br>
<img width="50%" src="assets/image-8.png"/>
<br>
<br>
If, on the other hand, the model's output is closer to 0, like in the second example, the prediction will be `0`.
<br>
<br>
<img width="50%" src="assets/image-10.png"/>
<br>
<br>
For a multi-class classification, the model outputs one probability per class and we take the highest probability as the model's prediction.
<br>
In this case, the highest probability is stored in the third neuron, so the prediction is `2`.

This exercise is about binary classification though, so the `predict()` method must simply output 0 or 1 depending on the input value.

In [ ]:
def predict(self, x: np.ndarray):
    """
    The predict method returns its predictions for the given input data.
    """
    pass

MyLogisticRegression.predict = predict

There are two more methods which you should already be familiar with.

Indeed, the `_optimize()` and `_linear()` methods are the same as in Linear Regression.

Go ahead and adapt them to our LogisticRegression class :

In [ ]:
def linear(self, x: np.ndarray):
    """
    The linear transformation applies the layer's weights and biases to the input data.
    """
    pass

MyLogisticRegression._linear = linear

def optimize(self, dw: np.ndarray, db: np.ndarray):
    """
    The optimize method performs the gradient descent update to the weights and biases.
    """
    pass

MyLogisticRegression._optimize = optimize

## Testing ##
test_mdl = MyLogisticRegression()
test_mdl.fit(x_train, y_train, train=False)
assert test_mdl._linear(np.array([1, 2])) == 0, "the linear transformation is implemented correctly"
test_mdl._optimize(np.array([1, 2]), 3)
assert np.mean(test_mdl.w) == np.mean(- np.array([1,2]) * LR), "the weights are not updated correctly"
assert test_mdl.b == - 3 * LR, "the bias is not updated correctly"
assert test_mdl._linear(np.array([1, 2])) == -0.8, "the linear transformation is implemented correctly"

Great, now there are two parts of the neural network left to implement :

* The `forward()` pass, which requires the implementation of a `_sigmoid()` method
* The `backward()` pass, which requires the implementation of the binary cross entropy (`_bce()`) loss

Let's start with `sigmoid()` because it's easier :

The sigmoid function is what makes our output layer serve as a probability of our value belonging to one or the other class because it squashes its input between 0 and 1.

$$ S(x) = \frac{1}{1 + e^{-x}} $$

In [ ]:
@staticmethod
def sigmoid(x: np.ndarray):
    """
    The sigmoid function squashes the input data between 0 and 1.
    """
    pass
    
MyLogisticRegression._sigmoid = sigmoid

## Testing ##
x_sigmoid = np.arange(-10, 10, 0.1)
plt.plot(x_sigmoid, [sigmoid(x) for x in x_sigmoid])

If you've implemented sigmoid correctly, you should recognize matplotlib's output if you do a [quick google search](https://www.google.com/search?sca_esv=593424282&rlz=1C5CHFA_enFR1086FR1086&sxsrf=AM9HkKnhFXQw46XVx7yP5nyzZOxkebfGWw:1703424283700&q=sigmoid&tbm=isch&source=lnms&sa=X&sqi=2&ved=2ahUKEwiXptP6laiDAxVBU6QEHVYpCxIQ0pQJegQIDhAB&biw=1512&bih=738&dpr=2) for sigmoid !

With this we can implement our `forward()` method :

In [ ]:
def forward(self, x: np.ndarray):
    """
    The forward method passes the input data through the model's transformations.
    """
    pass

MyLogisticRegression._forward = forward
## Testing ##
test_mdl = MyLogisticRegression()
test_mdl.fit(x_train, y_train, train=False)
test_mdl._optimize(np.array([-6, 2]), 3)
outputs = test_mdl._forward(np.array([[1,2], [3,4]]))
assert np.mean(np.round(outputs)) == 0.5, "the forward method is not implemented correctly"

We are calculating derivatives using the Binary Cross Entropy loss function.

$$ BCE =  - \frac{1}{N} \sum_{i=0}^N y_i * log(\overline{y}_i) + (1 - y_i) * log(1 - \overline{y}_i)

Note that this seemingly complex formula is just an **elegant** way of saying :

```python
if y_i == 1:
    return log(1 - ypred_i)
else if y_i == 0:
    return log(ypred_i)
```

Because we're in a binary classifcation, so our classes are either 0 or 1; it means that :
* if we multiply something with with $ 1 - y_i $, it will be ignored when $y_i$ equals 1\
* and vice versa for multiplying by $ y_i $ when it equals 0.

> This explanation is just to make the formula less intimidating\
> We recommend that you implement the formula as is, no need to deconstruct it into `if` statements

Go ahead and implement the BCE function below :

* Remember how we turned $ \frac{1}{N} \sum_{i=0}^N $ into code for linear regression
* To avoid multiplying by zero, add a very small value to `y_pred` each time you use it
> tip: you can do some really neat things with numbers in python\
> for example, you can easily use scientific notation by writing $xe-n$ (python example below:)

In [ ]:
assert 1e-2 == 0.01
1e-9

In [ ]:
def bce(self, y_pred: np.ndarray, y: np.ndarray):
    """
    Binary Cross Entropy is a method of calculating the model's loss.
    """
    pass

MyLogisticRegression._bce = bce

For the backward, we'll spare you the math, but here are some resources if you wish to do it on your own and you get stuck or have questions:

* https://www.python-unleashed.com/post/derivation-of-the-binary-cross-entropy-loss-gradient
* https://math.stackexchange.com/a/3220477

The partial derivative of $BCE$ with respect to $w$ is :

$$ x^T * (\overline{y} - y) $$

And the partial derivative w.r.t. $b$ is :

$$ \overline{y} - y $$

Write a `backward()` method which computes the gradients using these formulas.

* Remember that we are handling multiple inputs so you can retrieve the average of the gradients.

In [ ]:
def backward(self, x: np.ndarray, y: np.ndarray):
    """
    The backward method calculates the gradients for the weights and biases.
    """
    pass

MyLogisticRegression._backward = backward

We didn't write tests for the above methods because we've actually reached the end of our neural network !

We have all the building blocks to train and predict using our model !

Run the code below (which you'll find is very similar to the one we used earlier with sklearn's methods) and compare your outputs to sklearn's !

In [ ]:
mdl = MyLogisticRegression()
mdl.fit(x_train, y_train)
# !!! There's a chance you might encounter a RunTimeWarning error here !!!
# This is because your sigmoid function overflows
# The training loop should still work, but you can look online for ways to fix this error if you want !

plt.scatter(x_train[:,0], x_train[:,1], c=mdl.predict(x_train))
plt.show()

print(f"{(mdl.predict(x_train) == y_train).mean()*100}% accuracy")

If you've got an accuracy of over 95%, it means you've done pretty well !

All of this is cute and all, but writing all these functions by hand and computing the derivatives is kind of annoying, isn't it ?

Sure, sklearn provides nice ready-for-use functions and we haven't really looked into the countless ways we can modify their functions to our liking, but surely there must be a middle ground... A tool that spares us most of the formulas and doesn't require us to compute derivatives by ourselves each time we change something in our model's architecture...

We'll introduce such a tool in the final module of the day !